# Aircraft Sightings Over Time

This notebook calculates and exports:

- Daily and daily rolling average aircraft sightings over time
- Monthly sightings over time
- Yearly sightings over time

In [1]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [2]:
# Window for daily rolling average calculation
window = 7

# Number of days/months to represent in the daily/monthly charts
number_of_days = 365
number_of_months = 12

In [3]:
import matplotlib.pyplot as plt

def export_daily_chart(export_folder_path, daily, daily_rolling, number_of_days):
    # Plot the daily statistics
    fig = plt.figure(figsize=(14, 5))
    plt.plot(daily.index, daily.values, label="Daily Sightings", alpha=0.5)
    plt.plot(daily_rolling.index, daily_rolling.values, label=f"{window}-Day Rolling Average", color='red')
    plt.title(f"{window}-Day Rolling Average Aircraft Sightings per Day ({number_of_days} Days)")
    plt.xlabel("Date")
    plt.ylabel("Number of Sightings")
    plt.legend()
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"sightings-over-time-daily", "png")

    # Close the plot
    plt.close(fig)

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

def export_monthly_chart(export_folder_path, data, number_of_months):
    # The monthly data index has been converted to a timestamp, above, for export
    x = np.arange(len(data.index))  # numerical x-axis for regression
    y = data.values

    # Calculate the monthly trend line
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    trend = slope * x + intercept

    # Plot sightings by month
    fig = plt.figure(figsize=(14, 5))
    plt.bar(data.index, y, width=20, label="Monthly Sightings", color="cornflowerblue")
    plt.plot(data.index, trend, label="Trend Line", linestyle="--", color="red")
    plt.title(f"Aircraft Sightings by Month ({number_of_months} Months)")
    plt.xlabel("Month")
    plt.ylabel("Number of Sightings")
    plt.legend()
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"sightings-over-time-monthly", "png")

    # Close the plot
    plt.close(fig)

In [5]:
import matplotlib.pyplot as plt

def export_yearly_chart(export_folder_path, data):
    # Plot Sightings by Year with Trend Line
    x = data.index.values
    y = data.values
    slope, intercept, _, _, _ = linregress(x, y)

    fig = plt.figure(figsize=(10, 5))
    plt.bar(x, y, color="steelblue", width=0.6, label="Yearly Sightings")
    plt.plot(x, slope * x + intercept, color="red", linestyle="--", label="Trend Line")
    plt.title("Aircraft Sightings by Year")
    plt.xlabel("Year")
    plt.ylabel("Number of Sightings")
    plt.legend()
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, f"sightings-over-time-yearly", "png")

    # Close the plot
    plt.close(fig)

In [6]:
import pandas as pd

# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Aggregate sightings
    daily = df.groupby(df["Date"].dt.date).size().rename("Sightings")
    monthly = df.groupby(df["Date"].dt.to_period("M")).size().rename("Sightings")
    yearly = df.groupby(df["Date"].dt.year).size().rename("Sightings")

    # Calculate a roling average for the daily data
    daily_rolling = daily.rolling(window=window).mean()

    # Convert the monthly PeriodIndex to Timestamp
    monthly.index = monthly.index.to_timestamp()

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "sightings-over-time-daily.xlsx", {
        "Daily": daily.to_frame(name="Sightings"),
        "Daily Rolling Average": daily_rolling.to_frame(name="Sightings")
    })
    export_to_spreadsheet(export_folder_path, "sightings-over-time-monthly.xlsx", {
        "Monthly": monthly.to_frame(name="Sightings")
    })
    export_to_spreadsheet(export_folder_path, "sightings-over-time-yearly.xlsx", {
        "Yearly": yearly.to_frame(name="Sightings")
    })

    # Create a copy of the daily counts that only includes the last "N" days, for charting
    daily.index = pd.to_datetime(daily.index)
    current_day = pd.Timestamp.today().to_period("D").to_timestamp()
    start_day = current_day - pd.DateOffset(days=number_of_days-1)
    daily_last_n_days = daily.loc[start_day:current_day].copy()

    # Repeat for the daily rolling average
    daily_rolling.index = pd.to_datetime(daily_rolling.index)
    daily_rolling_last_n_days = daily_rolling.loc[start_day:current_day].copy()

    # Create a copy of the monthly counts that only includes the last "N" months, for charting
    monthly.index = pd.to_datetime(monthly.index)
    current_month = pd.Timestamp.today().to_period("M").to_timestamp()
    start_month = current_month - pd.DateOffset(months=number_of_months-1)
    monthly_last_n_months = monthly.loc[start_month:current_month].copy()

    # Export the charts
    export_daily_chart(export_folder_path, daily_last_n_days, daily_rolling_last_n_days, number_of_days)
    export_monthly_chart(export_folder_path, monthly_last_n_months, number_of_months)
    export_yearly_chart(export_folder_path, yearly)

except ValueError:
    print(f"WARNING: No data found")